In [280]:
#directly using substring from date seems causes subsequent failure when mereging.


import os.path as path
import pandas as pd

In [281]:
input_path="data"
air_reserve=pd.read_csv(path.join(input_path,"air_reserve.csv"))
air_reserve["reserve_date"]= air_reserve.apply(lambda row:row["reserve_datetime"][0:10], axis=1)
air_reserve["reserve_date"]=pd.to_datetime(air_reserve["reserve_date"])
air_reserve["reserve_date"]=air_reserve["reserve_date"].dt.date
air_reserve_group=air_reserve.groupby(["air_store_id", "reserve_date"], as_index=False)["reserve_visitors"].sum()
air_reserve_group.head()

,air_store_id,reserve_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-01,9
2,air_00a91d42b08b08d9,2016-12-08,18
3,air_00a91d42b08b08d9,2016-12-11,2
4,air_00a91d42b08b08d9,2016-12-18,4


In [282]:
air_visit=pd.read_csv(path.join(input_path,"air_visit_data.csv"))
air_store_info=pd.read_csv(path.join(input_path,"air_store_info.csv"))
date_info= pd.read_csv(path.join(input_path,"date_info.csv"))
store_id_relation=pd.read_csv(path.join(input_path,"store_id_relation.csv"))

In [344]:
air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [283]:
#format date
hpg_reserve= pd.read_csv(path.join(input_path,"hpg_reserve.csv"))
hpg_reserve["reserve_date"]=hpg_reserve.apply(lambda row:row["reserve_datetime"][0:10], axis=1)
hpg_reserve["reserve_date"]=pd.to_datetime(hpg_reserve["reserve_date"])
hpg_reserve["reserve_date"]=hpg_reserve["reserve_date"].dt.date
hpg_reserve_group=hpg_reserve.groupby(["hpg_store_id", "reserve_date"],as_index=False)["reserve_visitors"].sum()
hpg_reserve_group.head()

,hpg_store_id,reserve_date,reserve_visitors
0,hpg_001112ef76b9802c,2016-02-09,9
1,hpg_001112ef76b9802c,2016-03-03,3
2,hpg_001112ef76b9802c,2016-03-25,5
3,hpg_001112ef76b9802c,2016-04-01,13
4,hpg_001112ef76b9802c,2016-04-15,9


In [284]:
date_info.day_of_week=date_info.day_of_week.astype("category")
date_info.day_of_week=date_info.day_of_week.cat.codes

In [285]:
hpg_store_info=pd.read_csv(path.join(input_path,"hpg_store_info.csv"))

In [286]:
##hpg_reserve is a much bigger database

len(air_reserve)

92378

In [287]:
len(hpg_reserve)

2000320

In [288]:
air_visit.visit_date=pd.to_datetime(air_visit.visit_date)
air_visit.visit_date=air_visit.visit_date.dt.date
air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [289]:
m1=pd.merge(air_visit, air_reserve_group, left_on=["air_store_id","visit_date"], right_on=["air_store_id","reserve_date"],how='left').drop(['reserve_date'],axis=1)

In [290]:
m1.head()

,air_store_id,visit_date,visitors,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,NaN


In [291]:
#only a few have reserve data
m1.reserve_visitors.isnull().value_counts()

True     223599
False     28509
Name: reserve_visitors, dtype: int64

In [292]:
#that's about right
len(air_reserve_group)

31680

In [293]:
m2=pd.merge(store_id_relation, hpg_reserve_group, on='hpg_store_id')[["air_store_id","reserve_date","reserve_visitors"]]
m2.head()

,air_store_id,reserve_date,reserve_visitors
0,air_63b13c56b7201bd9,2016-01-01,3
1,air_63b13c56b7201bd9,2016-01-02,4
2,air_63b13c56b7201bd9,2016-01-03,7
3,air_63b13c56b7201bd9,2016-01-07,13
4,air_63b13c56b7201bd9,2016-01-08,14


In [294]:
#829 stores in air_visit. of them 150 store have hpg store info, which then have hpg reserve data
print(len(store_id_relation))
print(len(air_store_info))
print(len(hpg_store_info))
print(air_visit.air_store_id.value_counts()) 

150
829
4690
air_5c817ef28f236bdf    477
air_a083834e7ffe187e    476
air_36bcf77d3382d36e    476
air_d97dabf7aae60da5    476
air_71903025d39a4571    475
air_232dcee6f7c51d37    475
air_60a7057184ec7ec7    475
air_7a946aada80376a4    474
air_883ca28ef0ed3d55    474
air_4cca5666eaf5c709    473
air_d0e8a085d8dc83aa    473
air_cb7467aed805e7fe    473
air_04341b588bde96cd    472
air_fe22ef5a9cbef123    472
air_6b15edd1b4fbb96a    472
air_dea0655f96947922    472
air_3c05c8f26c611eb9    472
air_7cf5a02c0e01b647    471
air_39dccf7df20b1c6a    471
air_ba495cccc8f0f237    471
air_e270aff84ac7e4c8    470
air_2b8b29ddfd35018e    470
air_638c35eb25e53eea    470
air_f6b2489ccf873c3b    470
air_640cf4835f0d9ba3    470
air_287d2de7d3c93406    470
air_536043fcf1a4f8a4    470
air_8c119d1f16049f20    470
air_93ebe490d4abb8e9    469
air_383f5b2f8d345a49    469
                       ... 
air_e00fe7853c0100d6    149
air_1f34e9beded2231a    148
air_0ead98dd07e7a82a    138
air_fb44f566d4f64a4e    134
air_ef4

In [295]:
m3=pd.merge(m1,m2,left_on=["air_store_id",'visit_date'],right_on=["air_store_id","reserve_date"],how="left").drop("reserve_date",axis=1)

In [296]:
m3.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN


In [297]:
m3.reserve_visitors_y.isnull().value_counts()

True     238179
False     13929
Name: reserve_visitors_y, dtype: int64

In [298]:
m4=pd.merge(m3,air_store_info,on='air_store_id',how='left')
m4.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude,longitude
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [299]:
date_info.calendar_date=pd.to_datetime(date_info.calendar_date)
date_info.calendar_date=date_info.calendar_date.dt.date

In [300]:
m5=pd.merge(m4, date_info,left_on='visit_date',right_on='calendar_date',how='left').drop("calendar_date",axis=1)

In [301]:
m5.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0


In [302]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [303]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [304]:
m6=pd.merge(store_id_relation, hpg_store_info, on='hpg_store_id')

In [305]:
m6.head()

,air_store_id,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,air_a24bf50c3e90d583,hpg_c34b496d0305a809,Italian,Shizuoka-ken Hamamatsu-shi Sukenobuchō,34.721644,137.738944
1,air_a38f25e3399d1b25,hpg_50378da9ffb9b6cd,Japanese style,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
2,air_3c938075889fc059,hpg_349b1b92f98b175e,Japanese style,Fukuoka-ken Fukuoka-shi Ōmiya,33.579734,130.401762
3,air_ef789667e2e6fe96,hpg_f76ad2d2d0134bdd,International cuisine,Tōkyō-to Machida-shi Nakamachi,35.549670,139.448494
4,air_8f3b563416efc6ad,hpg_d12fb3b2b4129cd6,Creative Japanese food,Tōkyō-to Shinjuku-ku None,35.691384,139.701256


In [306]:
len(hpg_store_info)

4690

In [307]:
m7=pd.merge(m5,m6,on='air_store_id',how='left')

In [308]:
m7.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,holiday_flg,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0,NaN,NaN,NaN,NaN,NaN


In [309]:
len(m7)

252108

In [310]:
m7.hpg_store_id.isnull().value_counts()

True     232010
False     20098
Name: hpg_store_id, dtype: int64

In [311]:
#the result shows that a total of 829 store are there in the database, of which 150 has hpg_names. 


In [312]:
total_visitors=m7.groupby(["air_store_id"],as_index=False)["visitors"].sum()
total_visitors=total_visitors.rename(columns={"visitors":"total_visitors"})
total_visitors.head()

,air_store_id,total_visitors
0,air_00a91d42b08b08d9,6051
1,air_0164b9927d20bcc3,1378
2,air_0241aa3964b7f861,3919
3,air_0328696196e46f18,921
4,air_034a3d5b40d5b1b1,3722


In [313]:
m8=pd.merge(m7,total_visitors,on='air_store_id',how='left')
m8.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,holiday_flg,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0,NaN,NaN,NaN,NaN,NaN,8908
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,NaN,NaN,NaN,NaN,NaN,8908
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,NaN,NaN,NaN,NaN,NaN,8908
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,NaN,NaN,NaN,NaN,NaN,8908
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0,NaN,NaN,NaN,NaN,NaN,8908


In [314]:
#the reservation data is very limited?
m8['reserve_visitors_x'].isnull().value_counts()

True     223599
False     28509
Name: reserve_visitors_x, dtype: int64

In [315]:
m8['reserve_visitors_y'].isnull().value_counts()

True     238179
False     13929
Name: reserve_visitors_y, dtype: int64

In [316]:
len(air_reserve_group)

31680

In [317]:
len(hpg_reserve_group)

1444616

In [318]:
m8["year"]=pd.to_datetime(m8.visit_date).dt.year
m8["month"]=pd.to_datetime(m8.visit_date).dt.month
m8["day"]=pd.to_datetime(m8.visit_date).dt.day

In [319]:

m8.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,holiday_flg,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,15
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,16
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,18


In [320]:
m8.air_genre_name=m8.air_genre_name.astype("category")
m8.air_genre_name=m8.air_genre_name.cat.codes
m8.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,holiday_flg,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,15
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,16
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,18


In [321]:
#air store information is very complete
m8["air_area_name"].isnull().value_counts()

False    252108
Name: air_area_name, dtype: int64

In [322]:
m8.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,holiday_flg,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,15
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,16
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,0,NaN,NaN,NaN,NaN,NaN,8908,2016,1,18


In [323]:
#aggregate store visitors by month
month_visitor_group=m8.groupby(["air_store_id","year","month"],as_index=False)["visitors"].sum()
month_visitor_group=month_visitor_group.rename(columns={"visitors":"month_visitors"})
month_visitor_group.head()

,air_store_id,year,month,month_visitors
0,air_00a91d42b08b08d9,2016,7,652
1,air_00a91d42b08b08d9,2016,8,580
2,air_00a91d42b08b08d9,2016,9,548
3,air_00a91d42b08b08d9,2016,10,558
4,air_00a91d42b08b08d9,2016,11,543


In [324]:
m9=pd.merge(m8,month_visitor_group,on=['air_store_id','year','month'],how='left')
m9.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,...,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day,month_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,6,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,13,328
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,14,328
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,15,328
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,16,328
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,18,328


In [325]:
#aggregate store visit by day
m9.groupby(["day_of_week"])["visitors"].mean()

day_of_week
0    23.072737
1    17.177009
2    26.313688
3    23.873362
4    18.922702
5    17.672137
6    19.230121
Name: visitors, dtype: float64

In [326]:
m9.air_area_name.value_counts()

Fukuoka-ken Fukuoka-shi Daimyō                   19775
Tōkyō-to Shibuya-ku Shibuya                      17352
Tōkyō-to Minato-ku Shibakōen                     14696
Tōkyō-to Shinjuku-ku Kabukichō                   12517
Tōkyō-to Setagaya-ku Setagaya                     8719
Tōkyō-to Chūō-ku Tsukiji                          8370
Ōsaka-fu Ōsaka-shi Ōgimachi                       7560
Hiroshima-ken Hiroshima-shi Kokutaijimachi        7250
Tōkyō-to Suginami-ku Asagayaminami                6882
Ōsaka-fu Ōsaka-shi Kyūtarōmachi                   6212
Tōkyō-to Meguro-ku Kamimeguro                     5991
Miyagi-ken Sendai-shi Kamisugi                    5959
Hokkaidō Sapporo-shi Minami 3 Jōnishi             5755
Tōkyō-to Chiyoda-ku Kudanminami                   5411
Tōkyō-to Taitō-ku Higashiueno                     5247
Hyōgo-ken Kōbe-shi Kumoidōri                      5233
Fukuoka-ken Fukuoka-shi Hakata Ekimae             5059
Tōkyō-to Chūō-ku Ginza                            4225
Shizuoka-k

In [327]:
m9.air_area_name=m9.air_area_name.astype("category")
m9.air_area_name=m9.air_area_name.cat.codes
m9.head()


,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,...,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day,month_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,4,62,35.658068,139.751599,6,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,13,328
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,4,62,35.658068,139.751599,4,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,14,328
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,4,62,35.658068,139.751599,0,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,15,328
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,4,62,35.658068,139.751599,2,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,16,328
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,4,62,35.658068,139.751599,1,...,NaN,NaN,NaN,NaN,NaN,8908,2016,1,18,328


In [328]:
#group by genre, area, and day of week 
group3=m9.groupby(["air_genre_name","air_area_name","day_of_week"],as_index=False)["visitors"].mean()
group3=group3.rename(columns={"visitors":"visitor-mean-by-genre"})
group3.head()

,air_genre_name,air_area_name,day_of_week,visitor-mean-by-genre
0,0,74,0,38.829545
1,0,74,1,33.964286
2,0,74,2,44.684783
3,0,74,3,47.533333
4,0,74,4,33.357143


In [329]:
m10=pd.merge(m9,group3,on=["air_genre_name","air_area_name","day_of_week"],how='left')
m10.head()

,air_store_id,visit_date,visitors,reserve_visitors_x,reserve_visitors_y,air_genre_name,air_area_name,latitude_x,longitude_x,day_of_week,...,hpg_genre_name,hpg_area_name,latitude_y,longitude_y,total_visitors,year,month,day,month_visitors,visitor-mean-by-genre
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,4,62,35.658068,139.751599,6,...,NaN,NaN,NaN,NaN,8908,2016,1,13,328,21.701754
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,4,62,35.658068,139.751599,4,...,NaN,NaN,NaN,NaN,8908,2016,1,14,328,21.032500
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,4,62,35.658068,139.751599,0,...,NaN,NaN,NaN,NaN,8908,2016,1,15,328,26.363861
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,4,62,35.658068,139.751599,2,...,NaN,NaN,NaN,NaN,8908,2016,1,16,328,23.770574
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,4,62,35.658068,139.751599,1,...,NaN,NaN,NaN,NaN,8908,2016,1,18,328,12.677019


In [330]:
pre_training=m10[["air_store_id","visit_date","visitors","visitor-mean-by-genre","month_visitors","year","month","day"]]

In [331]:
sample_submission=pd.read_csv(path.join(input_path,"sample_submission.csv"))
sample_submission["air_store_id"]=sample_submission.apply(lambda row: row["id"][0:20],axis=1)
sample_submission["visit_date"]=sample_submission.apply(lambda row: row["id"][21:31],axis=1)
sample_submission["visit_date"]=pd.to_datetime(sample_submission["visit_date"])
sample_submission["visit_date"]=sample_submission["visit_date"].dt.date
sample_submission.head()

,id,visitors,air_store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [332]:
t1=pd.merge(sample_submission, air_reserve_group, left_on=["air_store_id","visit_date"], right_on=["air_store_id","reserve_date"],how='left')

In [333]:
sample_submission.head()

,id,visitors,air_store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [334]:
air_reserve_group.head()

,air_store_id,reserve_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-01,9
2,air_00a91d42b08b08d9,2016-12-08,18
3,air_00a91d42b08b08d9,2016-12-11,2
4,air_00a91d42b08b08d9,2016-12-18,4


In [335]:
r=list(map(type,air_reserve_group.reserve_date))
import collections
collections.Counter(r)

Counter({datetime.date: 31680})

In [336]:
r2=list(map(type,sample_submission.visit_date))
import collections
collections.Counter(r2)

Counter({datetime.date: 32019})

In [337]:
#ok there is no reservationd data available for prediction date, which means reservation data is of no use?
t1.head()

,id,visitors,air_store_id,visit_date,reserve_date,reserve_visitors
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,NaN,NaN
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,NaN,NaN
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,NaN,NaN
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,NaN,NaN
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,NaN,NaN


In [338]:
#also no intersection with hpg reserve data.
s1=set(sample_submission.visit_date)
s2=set(hpg_reserve.reserve_date)
s1.intersection(s2)

set()

In [339]:
sample_submission["year"]= pd.to_datetime(sample_submission["visit_date"]).dt.year
sample_submission["month"]= pd.to_datetime(sample_submission["visit_date"]).dt.month
sample_submission["day"]= pd.to_datetime(sample_submission["visit_date"]).dt.day
sample_submission.head()

,id,visitors,air_store_id,visit_date,year,month,day
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,2017,4,27


In [340]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,0,1
1,2016-01-02,2,1
2,2016-01-03,3,1
3,2016-01-04,1,0
4,2016-01-05,5,0


In [341]:
sample_submission=pd.merge(sample_submission, date_info,left_on=["visit_date"],right_on=["calendar_date"],how='left').drop("calendar_date",axis=1)

In [342]:
sample_submission.head()

,id,visitors,air_store_id,visit_date,year,month,day,day_of_week,holiday_flg
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,2017,4,23,3,0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,2017,4,24,1,0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,2017,4,25,5,0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,2017,4,26,6,0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,2017,4,27,4,0


TypeError: merge() missing 1 required positional argument: 'right'